In [23]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

range_eco = 10
width_eco = 5
range_signal = 5
range_pred = 5

class Predator(Agent):
    def __init__(self, unique_id, model, pos, face):
        super().__init__(unique_id, model)
        self.pos = pos
        self.face = face # N = (0, 1), S = (0, -1), E = (1, 0), W = (-1, 0)
        self.alive = True
        self.preDir = None

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)
        
    def smell_prey(self):
        """ Search for whales in the all direction in its range (smell) """
        pass
    
    def eat(self):
        """ Eat any(1) whale on the same cell"""
        pass
    
    def step(self):
        self.move()

class Whale(Agent):
    """ A whale agent, which can use echolocation to detect objects and send signals to other whales. """
    def __init__(self, unique_id, model, pos, face):
        super().__init__(unique_id, model)
        self.pos = pos
        self.face = face # N = (0, 1), S = (0, -1), E = (1, 0), W = (-1, 0)
        self.alive = True
        self.preDir = None

    def move(self, new_position):
        self.model.grid.move_agent(self, new_position)
    
    def echolocation(self): 
        """ Send a signal in the direction it is currently facing. 
            Check for predators or food in that direction. 
            Returns the location of predator if found. """
        area_scanned = []
        x, y = self.pos
        dx, dy = self.face
        pred_coords = []
        for r in range(1, range_eco + 1): 
            for w in range (-1 * (width_eco//2), width_eco//2 + 1):
                if dx == 0: area_scanned.append((x + w, y + dy * r))
                else: area_scanned.append((x + dx * r, y + w))
        for cell in area_scanned:
            try:
                cellmates = self.model.grid.get_cell_list_contents([cell])
                for i in cellmates: 
                    if type(i) is Predator: pred_coords.append(cell)
            except: 
                pass
        if pred_coords: return pred_coords
        return False
    
    def step(self):
        self.move()
        if self.wealth > 0:
            self.give_money()

class Model(Model):
    """A model with some number of agents."""
    def __init__(self, N, width, height):
        self.num_agents = N
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        self.running = True

        # Create agents
        for i in range(self.num_agents):
            a = MoneyAgent(i, self)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))

        self.datacollector = DataCollector(
            agent_reporters={"Wealth": "wealth"})

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

In [2]:
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer

In [3]:
def agent_portrayal(agent):
    portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "r": 0.5}

    if agent.wealth > 0:
        portrayal["Color"] = "red"
        portrayal["Layer"] = 0
    else:
        portrayal["Color"] = "grey"
        portrayal["Layer"] = 1
        portrayal["r"] = 0.2
    return portrayal

In [4]:
grid = CanvasGrid(agent_portrayal, 10, 10, 500, 500)
server = ModularServer(MoneyModel,
                       [grid],
                       "Money Model",
                       {"N":100, "width":10, "height":10})
server.port = 8520 # The default
server.launch()

NameError: name 'MoneyModel' is not defined

Interface starting at http://127.0.0.1:8522


RuntimeError: This event loop is already running

Socket opened!
{"type":"reset"}
{"type":"get_step","step":1}


In [24]:
class Seti(Agent): 
    def __init__(self):
        pass
s = Seti()

In [30]:
type(s) is Seti

True